In [2]:
from parametros import tbl_excPot_med123,  tbl_excPot_med4y5, tbl_coeficiente_Kp, tbl_termPrecPot
import numpy as np

In [8]:
#Valores Iniciales
potC = [2, 2, 3, 4, 4, 5]
tarifa = '6_3TD'
tipo_punto = 4

In [ ]:
#Dividir la factura en Meses
#Buscar los meses sobre los que tenemos información / Ej: '2020-11$
def monthly_bill(df_data):
    list_months = df_data.Fecha.unique()
    list_uniq_months = [x[:-3] for x in list_months]
    set_unique_dates = set(list_uniq_months)
    list_unique_dates = list(set_unique_dates)

    return list_unique_dates.sort()


#IMP: list_unique_dates

In [4]:
def precio_exceso_potencia(tarifa, tipo_punto):
    if np.any(tipo_punto <= 3):
        price = tbl_excPot_med123[tbl_excPot_med123['Tarifa'] == tarifa]['Precio_exceso_potencia_(€/kW*día)'].item()
    else:
        price = tbl_excPot_med4y5[tbl_excPot_med4y5['Tarifa'] == tarifa]['Precio_exceso_potencia_(€/kW*día)'].item()
    return price

precio_exceso_potencia(tarifa, tipo_punto)

0.07458

In [7]:
def precio_coeficiente_Kp(tarifa, period):
    return tbl_coeficiente_Kp[tbl_coeficiente_Kp['Tarifa'] == tarifa][period].item()

precio_coeficiente_Kp(tarifa, 'P3')

0.553151

In [9]:
def facturacion_potencia(potC, tarifa):
    precio_potencia = []
    for i, number in enumerate(potC):
        string_comp = 'Periodo ' + str(i+1)
        item = tbl_termPrecPot[tbl_termPrecPot['Tarifa'] == tarifa][string_comp]
        result = number * item.item()
        precio_potencia.append(result)
        
    return round(sum(precio_potencia), 2)

facturacion_potencia(potC, tarifa)

#def facturacion_potencia_2(potC, tarifa):
#    precio_potencia = []
#    for i, number in enumerate(potC):
#        string_comp = 'Periodo ' + str(i+1)
#        item = tbl_termPrecPot[tbl_termPrecPot['Tarifa'] == tarifa][string_comp]
#        result = number * item.item()
#        precio_potencia.append(result)
#    
#    precio_fijo_potencia = round(sum(precio_potencia), 2)
#    df_ini_prefijpot = pd.Series(precio_fijo_potencia).repeat(len(list_unique_dates)).set_axis(list_unique_dates)
#
#    return df_ini_prefijpot

75.42

In [ ]:
def facturacion_EP(potC, tarifa, tipo_punto):

    termino_exceso_potencia = precio_exceso_potencia(tarifa, tipo_punto)
    periodos = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
    mesuario = {}

    if tipo_punto == 1 or 2 or 3:

        df_exceso_euro_mes = pd.DataFrame()

        for date in list_unique_dates:
            new_df = cups_n1[cups_n1["Fecha"].str.startswith(date)]
            new_df.reset_index(drop=True, inplace=True)
            list_exceso = []
            list_SinExceso = []

            for i, Pdj in enumerate(new_df.ActivaEntrante):
            
                num_PX = int(new_df.periodo[i][1:])
                Pcp = potC[(num_PX-1)]

                if Pdj > Pcp:
                    result_4y5 = 2 * (Pdj - Pcp) * termino_exceso_potencia
                elif Pdj < Pcp or Pdj == Pcp:
                    result_4y5 = 0

                list_exceso.append(result_4y5)

            df_exceso_euro_mes[date] = pd.Series(list_exceso)

        df_exceso_euro_mes = df_exceso_euro_mes.fillna(0)
        precio_mes = round(df_exceso_euro_mes.sum(axis='index'), 2)

        return precio_mes

    elif tipo_punto == 4 or 5:

        for date in list_unique_dates:
            list_mes = []

            for period in periodos:
                new_df = cups_n1[(cups_n1["Fecha"].str.startswith(date)) & (cups_n1['periodo'] == period)]
                new_df.reset_index(drop=True, inplace=True)
                list_exceso = []

                for i, Pdj in enumerate(new_df.ActivaEntrante):
                
                    num_PX = int(period[1:])
                    Pcp = potC[(num_PX-1)]

                    if Pdj > Pcp:
                        result_123 = (Pdj - Pcp) ** 2
                    elif Pdj < Pcp or Pdj == Pcp:
                        result_123 = 0

                    #list_SinExceso.append(result_123)
                    list_exceso.append(result_123)

                sumatorio_1 = sum(list_exceso)
                #1Formula
                fep = precio_coeficiente_Kp(tarifa, period) * termino_exceso_potencia * math.sqrt(sumatorio_1)
                list_mes.append(fep)

            mesuario[date] = round(int(sum(list_mes)), 2)
            
        return mesuario

Función Objetivo

In [ ]:
#1) Primera opción 
from scipy.optimize import minimize, rosen, rosen_der
#2)  Segunda Opción [GEKKO SOLUTION]
from gekko import GEKKO    
#3) Tercera Opción [Dual annealing global optimization for the ackley multimodal objective function]
from scipy.optimize import dual_annealing

Scipy Minimize

In [ ]:
def constraint1_ineq(x): 
    return x[1] - x[0] 

def constraint2_ineq(x): 
    return x[2] - x[1] 

def constraint3_ineq(x): 
    return x[3] - x[2] 

def constraint4_ineq(x): 
    return x[4] - x[3] 

def constraint5_ineq(x): 
    return x[5] - x[4] 

def constraint6_ineq(x): 
    return fop - objective(x)

# initial guesses
n = 6
potC = np.zeros(n)
potC[0] = 8
potC[1] = 10
potC[2] = 13
potC[3] = 14
potC[4] = 15
potC[5] = 15

# show initial objective
print('Initial Objective: ' + str(objective(potC)))

# optimize
b = (0.0, 500) 
bnds = (b, b, b, b, b, b)
con6_ineq = {'type': 'ineq', 'fun': constraint6_ineq}
con5_ineq = {'type': 'ineq', 'fun': constraint5_ineq}
con4_ineq = {'type': 'ineq', 'fun': constraint4_ineq}
con3_ineq = {'type': 'ineq', 'fun': constraint3_ineq}
con2_ineq = {'type': 'ineq', 'fun': constraint2_ineq}
con1_ineq = {'type': 'ineq', 'fun': constraint1_ineq}

cons = ([con1_ineq, con2_ineq, con3_ineq, con4_ineq, con5_ineq, con6_ineq])
solution = minimize(objective, potC, args= (), method='SLSQP',\
                    bounds=bnds, constraints=cons)
x = solution.x

# show final objective
print('Final Objective: ' + str(objective(x, tarifa, tipo_punto)))

print('Solution')
print('x1 = ' + str(x[0]))
print('x2 = ' + str(x[1]))
print('x3 = ' + str(x[2]))
print('x4 = ' + str(x[3]))
print('x5 = ' + str(x[4]))
print('x6 = ' + str(x[5]))

Gekko

In [ ]:
m = GEKKO()
x = m.Array(m.Var,4,value=1,lb=1,ub=5)
x1,x2,x3,x4 = x
# change initial values
x2.value = 5; x3.value = 5
m.Equation(x1*x2*x3*x4>=25)
m.Equation(x1**2+x2**2+x3**2+x4**2==40)
m.Minimize(x1*x4*(x1+x2+x3)+x3)
m.solve()
print('x: ', x)
print('Objective: ',m.options.OBJFCNVAL)

Scipy Dual Anneling

In [ ]:
# objective function

def objective(v):
    x, y = v
    return '''formula'''

# define range for input
r_min, r_max = -5.0, 5.0

# define the bounds on the search
bounds = [[r_min, r_max], [r_min, r_max]]

# perform the dual annealing search
result = dual_annealing(objective, bounds)

# summarize the result
print('Status : %s' % result['message'])
print('Total Evaluations: %d' % result['nfev'])

# evaluate solution
solution = result['x']
evaluation = objective(solution)
print('Solution: f(%s) = %.5f' % (solution, evaluation))